## **6. Scaling**

### **6.1 Create Scorecard**

In [1]:
#import library
import pandas as pd
import numpy as np

#import library for modeling
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression

import sys
sys.path.append("../src")
#load configuration
import utils

In [2]:
config_data = utils.config_load()
config_data

{'raw_dataset_path': '../data/raw/Loan_default.csv',
 'dataset_path': '../data/output/data.pkl',
 'predictors_set_path': '../data/output/predictors.pkl',
 'response_set_path': '../data/output/response.pkl',
 'train_path': ['../data/output/X_train.pkl', '../data/output/y_train.pkl'],
 'test_path': ['../data/output/X_test.pkl', '../data/output/y_test.pkl'],
 'data_train_path': '../data/output/training_data.pkl',
 'data_train_binned_path': '../data/output/bin_training_data.pkl',
 'crosstab_list_path': '../data/output/list_crosstab.pkl',
 'WOE_table_path': '../data/output/WOE_table.pkl',
 'IV_table_path': '../data/output/IV_table.pkl',
 'WOE_map_dict_path': '../data/output/WOE_map_dict.pkl',
 'X_train_woe_path': '../data/output/X_train_woe.pkl',
 'response_variable': 'Default',
 'test_size': 0.2,
 'numeric_col': ['Age',
  'Income',
  'LoanAmount',
  'MonthsEmployed',
  'NumCreditLines',
  'InterestRate',
  'LoanTerm',
  'DTIRatio'],
 'categoric_col': ['Education',
  'EmploymentType',
  'Ma

In [3]:
def scaling():
    """
    Assign score points to each attribute based on the best model's output

    Returns
    -------
    pandas.DataFrame: A table containing the characteristics, WOE values, parameter estimates, and score points

    This function calculates score points for each attribute based on the best logistic regression model and the Weight of Evidence (WOE) values
    It uses predefined references like PDO (Points to Double the Odds) and offset to transform the logistic regression model output into score points
    The resulting score points are stored in a table and saved to a file. The table includes the characteristics, WOE values, parameter estimates, and score points for each attribute
    """

    #define the references: score, odds, pdo
    pdo = config_data['pdo']
    score = config_data['score_ref']
    odds = config_data['odds_ref']

    #load the best model
    best_model_path = config_data['best_model_path']
    best_model = utils.pickle_load(best_model_path)

    #load the WOE table
    WOE_table_path = config_data['WOE_table_path']
    WOE_table = utils.pickle_load(WOE_table_path)

    #load the best model's estimates table
    best_model_summary_path = config_data['best_model_summary_path']
    best_model_summary = utils.pickle_load(best_model_summary_path)

    #calculate Factor and Offset
    factor = pdo/np.log(2)
    offset = score-(factor*np.log(odds))

    print('===================================================')
    print(f"Odds of good of {odds}:1 at {score} points score.")
    print(f"{pdo} PDO (points to double the odds of good).")
    print(f"Offset = {offset:.2f}")
    print(f"Factor = {factor:.2f}")
    print('===================================================')

    #define n = number of characteristics
    n = best_model_summary.shape[0] - 1

    #define b0
    b0 = best_model.intercept_[0]

    print(f"n = {n}")
    print(f"b0 = {b0:.4f}")

    #adjust characteristic name in best_model_summary_table
    numeric_col = config_data['numeric_col']
    for col in best_model_summary['Characteristic']:

        if col in numeric_col:
            bin_col = col + '_binned'
        else:
            bin_col = col

        best_model_summary.replace(col, bin_col, inplace = True) 

    #,erge tables to get beta/parameter estimate for each characteristic
    scorecards = pd.merge(left = WOE_table,
                          right = best_model_summary,
                          how = 'left',
                          on = ['Characteristic'])
    
    #define beta and WOE
    beta = scorecards['Estimate']
    WOE = scorecards['WOE']

    #calculate the score point for each attribute
    scorecards['Points'] = (offset/n) - factor*((b0/n) + (beta*WOE))
    scorecards['Points'] = scorecards['Points'].astype('int')

    #validate
    print('Scorecards table shape : ', scorecards.shape)
    
    #dump the scorecards
    scorecards_path = config_data['scorecards_path']
    utils.pickle_dump(scorecards, scorecards_path)

    return scorecards

In [4]:
#check the function
scorecards = scaling()
scorecards

Odds of good of 30:1 at 200 points score.
20 PDO (points to double the odds of good).
Offset = 101.86
Factor = 28.85
n = 15
b0 = -0.0026
Scorecards table shape :  (58, 5)


,Characteristic,Attribute,WOE,Estimate,Points
0,Age_binned,"(17.999, 28.0]",-0.641271,-1.063223,-12
1,Age_binned,"(28.0, 38.0]",-0.264161,-1.063223,-1
2,Age_binned,"(38.0, 49.0]",0.137337,-1.063223,11
3,Age_binned,"(49.0, 59.0]",0.499244,-1.063223,22
4,Age_binned,"(59.0, 69.0]",0.882110,-1.063223,33
5,Income_binned,"(14999.999, 42062.0]",-0.553031,-1.060490,-10
6,Income_binned,"(42062.0, 69093.0]",0.031519,-1.060490,7
7,Income_binned,"(69093.0, 96027.6]",0.182972,-1.060490,12
8,Income_binned,"(96027.6, 123007.0]",0.245767,-1.060490,14
9,Income_binned,"(123007.0, 149999.0]",0.284585,-1.060490,15


In [5]:
#calculate the min and max points for each characteristic
grouped_char = scorecards.groupby('Characteristic')
grouped_points = grouped_char['Points'].agg(['min', 'max'])
grouped_points

,min,max
Characteristic,,
Age_binned,-12,33
DTIRatio_binned,4,10
Education,2,10
EmploymentType,1,13
HasCoSigner,3,10
HasDependents,3,10
HasMortgage,4,9
Income_binned,-10,15
InterestRate_binned,-9,27


In [6]:
#calculate the min and max score from the scorecards
total_points = grouped_points.sum()
min_score = total_points['min']
max_score = total_points['max']

print(f"The lowest credit score = {min_score}")
print(f"The highest credit score = {max_score}")

The lowest credit score = -6
The highest credit score = 212


### **6.2 Predict the Credit Score**

In [7]:
def get_points_map_dict():
    """
    Get the Points mapping dictionary

    Returns:
    dict: A dictionary containing the points mapping for each attribute and characteristic

    This function generates a points mapping dictionary based on the scorecards table
    It iterates through the table, extracts the characteristics, attributes, and their corresponding points, and organizes them into a dictionary structure
    The resulting dictionary is then saved to a file
    """
    #load the Scorecards table
    scorecards = utils.pickle_load(config_data['scorecards_path'])

    #initialize the dictionary
    points_map_dict = {}
    points_map_dict['Missing'] = {}
    unique_char = set(scorecards['Characteristic'])
    for char in unique_char:
        #get the Attribute & WOE info for each characteristics
        current_data = (scorecards
                            [scorecards['Characteristic']==char]    
                            [['Attribute', 'Points']])              
        
        #get the mapping
        points_map_dict[char] = {}
        for idx in current_data.index:
            attribute = current_data.loc[idx, 'Attribute']
            points = current_data.loc[idx, 'Points']

            if attribute == 'Missing':
                points_map_dict['Missing'][char] = points
            else:
                points_map_dict[char][attribute] = points
                points_map_dict['Missing'][char] = np.nan

    #validate data
    print('Number of key : ', len(points_map_dict.keys()))

    #dump
    utils.pickle_dump(points_map_dict, config_data['points_map_dict_path'])

    return points_map_dict
    

In [8]:
#check the function
get_points_map_dict()

Number of key :  16


{'Missing': {'LoanAmount_binned': nan,
  'NumCreditLines_binned': nan,
  'HasMortgage': nan,
  'MonthsEmployed_binned': nan,
  'HasDependents': nan,
  'Income_binned': nan,
  'Education': nan,
  'DTIRatio_binned': nan,
  'MaritalStatus': nan,
  'Age_binned': nan,
  'HasCoSigner': nan,
  'LoanPurpose': nan,
  'EmploymentType': nan,
  'LoanTerm_binned': nan,
  'InterestRate_binned': nan},
 'LoanAmount_binned': {Interval(4999.999, 53711.4, closed='right'): 18,
  Interval(53711.4, 103028.0, closed='right'): 13,
  Interval(103028.0, 151975.2, closed='right'): 8,
  Interval(151975.2, 201339.8, closed='right'): 2,
  Interval(201339.8, 249999.0, closed='right'): -3},
 'NumCreditLines_binned': {Interval(0.999, 2.0, closed='right'): 9,
  Interval(2.0, 3.0, closed='right'): 5,
  Interval(3.0, 4.0, closed='right'): 2},
 'HasMortgage': {'No': 4, 'Yes': 9},
 'MonthsEmployed_binned': {Interval(-0.001, 24.0, closed='right'): -5,
  Interval(24.0, 48.0, closed='right'): 1,
  Interval(48.0, 72.0, closed=

In [9]:
def transform_points(raw_data=None, type=None, config_data=None):
    """
    Replace data value with points

    Args
    ----
    raw_data (DataFrame, optional): The raw data to be transformed. If None, the data is loaded based on the specified 'type'
    type (str, optional): The type of data to be transformed (e.g., 'train', 'test'). If None, 'raw_data' must be provided
    config_data (dict, optional): Configuration data containing file paths and settings

    Returns
    -------
    DataFrame: The transformed data with values replaced by points

    This function replaces the values in the input data with their corresponding points based on the 'points_map_dict'
    It handles both numeric and categorical columns, mapping them to their respective points
    Missing or out-of-range values are also mapped to points
    The transformed data is returned, and if a 'type' is specified, it is saved to a file
    """
    #lLoad the numerical columns
    num_cols = config_data['numeric_col']

    #load the points_map_dict
    points_map_dict = utils.pickle_load(config_data['points_map_dict_path'])

    #load the saved data if type is not None
    if type is not None:
        raw_data = utils.pickle_load(config_data[f'{type}_path'][0])

    #map the data
    points_data = raw_data.copy()
    for col in points_data.columns:
        if col in num_cols:
            map_col = col + '_binned'
        else:
            map_col = col    

        points_data[col] = points_data[col].map(points_map_dict[map_col])

    #map the data if there is a missing value or out of range value
    for col in points_data.columns:
        if col in num_cols:
            map_col = col + '_binned'
        else:
            map_col = col 

        points_data[col] = points_data[col].fillna(value=points_map_dict['Missing'][map_col])

    #dump data
    if type is not None:
        utils.pickle_dump(points_data, config_data[f'X_{type}_points_path'])

    return points_data

In [10]:
#check the function on the train set
X_train_points = transform_points(type='train', config_data=config_data)

X_train_points

,Age,Income,LoanAmount,MonthsEmployed,NumCreditLines,InterestRate,LoanTerm,DTIRatio,Education,EmploymentType,MaritalStatus,HasMortgage,HasDependents,LoanPurpose,HasCoSigner
15826,11,-10,-3,-5,9,27,6,4,2,1,10,4,10,6,3
147371,11,15,13,21,5,8,6,6,10,7,10,9,10,6,10
178180,11,14,18,21,5,27,6,4,2,7,6,9,10,4,10
126915,11,-10,18,-5,5,0,6,4,5,7,10,4,10,6,3
163930,-12,14,-3,14,5,0,6,6,5,5,4,4,3,6,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59108,-12,14,8,8,5,27,6,8,9,1,4,4,3,4,10
71610,33,-10,13,14,2,27,6,4,10,1,10,4,3,6,10
85645,33,7,18,1,9,8,6,4,2,7,6,4,3,6,3
21010,-1,7,13,-5,9,0,6,10,2,5,4,9,10,6,3


In [11]:
def predict_score(raw_data, config_data):
    """
    Predict the credit score for a given dataset.

    Args
    ----
    raw_data (DataFrame): The raw data for which to predict the credit score.
    config_data (dict): Configuration data containing file paths and settings.

    Returns
    -------
    int: The predicted credit score.

    This function takes raw data as input, transforms it into points using the 'transform_points' function, and calculates the credit score by summing the points for each row
    The cutoff score specified in the configuration is used to make a recommendation (APPROVE or REJECT), and the predicted score is saved to a file
    """
    
    points = transform_points(raw_data = raw_data, 
                              type = None, 
                              config_data = config_data)
    
    score = int(points.sum(axis=1))
    
    cutoff_score = config_data['cutoff_score']

    if score > cutoff_score:
        print("Recommendation : APPROVE")
    else:
        print("Recommendation : REJECT")

    utils.pickle_dump(score, config_data['score_path'])

    return score


In [12]:
input = {
    'Age_binned': 32,
    'DTIRatio_binned': 0.23,
    'Education': "Master's",
    'EmploymentType': 'Part-Time',
    'HasCoSigner': 'Yes',
    'HasDependents': 'No',
    'HasMortgage': 'No',
    'Income_binned': 69000,
    'InterestRate_binned': 4.21,
    'LoanAmount_binned': 50610,
    'LoanPurpose': 'Other',
    'LoanTerm_binned': 12,
    'MaritalStatus': 'Divorced',
    'MonthsEmployed_binned': 8,
    'NumCreditLines_binned': 2
}

input = pd.DataFrame(input, index=[0])

input

,Age_binned,DTIRatio_binned,Education,EmploymentType,HasCoSigner,HasDependents,HasMortgage,Income_binned,InterestRate_binned,LoanAmount_binned,LoanPurpose,LoanTerm_binned,MaritalStatus,MonthsEmployed_binned,NumCreditLines_binned
0,32,0.23,Master's,Part-Time,Yes,No,No,69000,4.21,50610,Other,12,Divorced,8,2


In [13]:
#predict the credit score
predict_score(raw_data=input, config_data=config_data)

Recommendation : REJECT


107